<a href="https://colab.research.google.com/github/nicholasfurl/Great-Courses/blob/main/Lesson15_vgg16CNN_bugs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##########
#-----vgg16 DCNN bug versus animal classification
#This notebook trains an image classification model and makes gradual improvements, 
#debugging the model, to improve performance. GPUs are encouraged. 
#In colab, one can add a GPU by clicking the Runtime menu and selecting Change runtime type. 
#Selecting GPU as the hardware accelerator will allow for the usage of a GPU.
#########

In [1]:
#Download tinyImageNet
#200 categories, 500 images per category, images 64*64*3

#unzipping creates a tiny-imagenet-200 folder in active directory with test, train and val subfolders and two *.txt files
#Train subfolders have lots of further subfolders, eacj labveled n10*. Within each is some text files and an images folder
#containing n*.JPEG images
#Test subfolder has subfolder images with test_*JPEG files
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

--2023-01-03 16:30:48--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  20.1MB/s    in 15s     

2023-01-03 16:31:03 (16.0 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [3]:
!pip install keras=='2.8.0'
#from PIL import Image    #I knpow from previous lessons that this doesn't work
import keras.utils as image #This is the one that works
from keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.8.0 which is incompatible.


In [2]:
#specify folder names  (for training set?)
cats_0 = ['n01443537','n01629819','n01641577','n01644900','n01698640','n01742172',
          'n01855672','n01882714','n02002724','n02056570','n02058221','n02074367',
          'n02085620','n02094433','n02099601','n02099712','n02106662','n02113799',
          'n02123045','n02123394','n02124075','n02125311','n02129165','n02132136',
          'n02364673','n02395406','n02403003','n02410509','n02415577','n02423022',
          'n02437312','n02480495','n02481823','n02486410','n02504458','n02509815']
cats_1 = ['n01770393','n01774384','n01774750','n01784675','n02165456','n02190166',
          'n02206856','n02226429','n02231487','n02233338','n02236044','n02268443',
          'n02279972','n02281406']

In [20]:
#function def to read files and return training and test sets

#the cats argument will be one of the 
def read_cats(cats, lab, train_size, test_size):

  #initialise stuff for loop
  vecs = []
  labs = []

  num_images_per_folder = 500 #a bit more explanatory and robust than it was

  #loop through each folder name (a subcategory of animals or bugs) and then through the images in that folder
  for c in cats:  #folder names from argument
    for i in range(num_images_per_folder): #num images per folder`

    #load the image and convert it to an array
      img = image.load_img("tiny-imagenet-200/train/"+c+"/images/"+c+"_"+str(i)+".JPEG")
      img_arr = image.img_to_array(img)
      

      #vectorise then reshape? This appears to vectorise it and then just put it back as it was again! Why?
      #print(c, i, img_arr.shape) 
      img_arr = img_arr.flatten() #this is numpy not image
      #print(c, i, img_arr.shape)
      img_arr = img_arr.reshape(64,64,3)
      #print(c, i, img_arr.shape)

      
      vecs += [img_arr] #Accumulates 2D arrays? Or should it be vectorised?
      labs += [lab]   #But this will just be [] every time? Something's not right here ....

  vecs = np.asarray(vecs) #convert to array
  return(train_test_split(vecs,labs, train_size=train_size,test_size=test_size))

In [26]:
X0_train, X0_test, y0_train, y0_test = read_cats(cats_0, 0, .1, .2) #So hand it the animals (category 0), make its label 0, and train size will be .1q and test_size will be .2. Why pass these last two just to return them again? No idea, seems like unncessary complexity
X1_train, X1_test, y1_train, y1_test = read_cats(cats_1, 1, .1, .2)
#print(X0_train.shape) #1800*64*64*3 is what I want? According to the lecture, yes.
#print(y0_train)  #1800 0's is what I want? Yes, you gave lab as an argument so one run will be handed 0o and the other 1.

#Put all training examples together and all test examples together
X_train = np.concatenate((X0_train, X1_train))
X_test = np.concatenate((X0_test, X1_test))
y_train = np.concatenate((y0_train, y1_train))
y_test = np.concatenate((y0_test, y1_test))

In [ ]:
#Built vgg model!